In [ ]:
import pandas as pd

In [ ]:
articles = pd.read_parquet("data/ebnerd_large/articles.parquet")
train_hist = pd.read_parquet("data/ebnerd_large/train/history.parquet")
train_behaviors = pd.read_parquet("data/ebnerd_large/train/behaviors.parquet")
# val_hist = pd.read_parquet("data/ebnerd_large/validation/history.parquet")
# val_behaviors = pd.read_parquet("data/ebnerd_large/validation/behaviors.parquet")

In [ ]:
articles.head()#['subcategory'].nunique()

In [ ]:
articles['ner_clusters']

In [ ]:
train_behaviors.head()

In [ ]:
len(train_behaviors)

In [ ]:
train_hist.head()

In [ ]:
train_hist['user_id'].nunique()

### validation

In [ ]:
# val_hist = pd.read_parquet("data/ebnerd_large/validation/history.parquet")
val_behaviors = pd.read_parquet("data/ebnerd_large/validation/behaviors.parquet")

In [ ]:
len(val_behaviors)

### test

In [ ]:
# test_hist = pd.read_parquet("data/ebnerd_large/test/history.parquet")
test_behaviors = pd.read_parquet("data/ebnerd_large/test/behaviors.parquet")

In [ ]:
len(test_behaviors)

### TransE

In [ ]:
import pickle
with open("data/transe_wikidata5m.pkl", "rb") as fin:
    model = pickle.load(fin)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [ ]:
entity2id
# relation2id
# entity_embeddings
# relation_embeddings

In [ ]:
print("hellow")

In [1]:
from pandas import read_parquet, to_numeric
from tqdm import tqdm

In [2]:
articles = read_parquet("GLORY/data/ebnerd_small/articles.parquet")

In [3]:
art_dict = {}

for idx, line in tqdm(articles.iterrows(), desc=f"[Processing raw news"):
    # print(line)
    news_id = line['article_id']
    title = line['title']

    art_dict[news_id] = title

    # print(f"{idx}. {news_id} : {title}")

[Processing raw news: 20738it [00:00, 21142.41it/s]


In [4]:
hist = read_parquet("GLORY/data/ebnerd_small/train/history.parquet")
behaviors = read_parquet("GLORY/data/ebnerd_small/train/behaviors.parquet")

In [6]:
lines = []

In [7]:
edge_list, user_set = [], set()

In [ ]:
# for _,line in tqdm(hist.iterrows()):
    # print(line)

In [8]:
hist_art = set()

In [9]:
for _,line in tqdm(behaviors.iterrows()):
    # print(line)
    iid = line['impression_id']
    uid = line['user_id']
    time = line['impression_time']
    history = hist[hist['user_id'] == uid]['article_id_fixed'].values[0]
    [hist_art.add(h) for h in history]
    imp = list(line['article_ids_inview'])
    click = line['article_ids_clicked']
    click_one_hot = [0] * len(imp)
    click_id = [imp.index(c) for c in click]
    for c in click_id:
        click_one_hot[c] = 1
    impressions = list(zip(imp, click_one_hot))
    new_line = '\t'.join([str(iid), str(uid), str(time), ' '.join(str(h) for h in history), ' '.join('-'.join([str(id), str(i)]) for id,i in impressions)]) + '\n'
    lines.append(new_line)

232887it [02:20, 1659.99it/s]


In [11]:
len(art_dict), len(hist_art)

(20738, 8786)

In [10]:
y = 0
n = 0
for h in hist_art:
    if art_dict[h]:
        y += 1
    else:
        n += 1
        # print(f"{h}")

y, n


(8783, 3)

In [15]:
art_dict[int('9748432')]

'Frygter cyberangreb under Eurovision'

In [12]:
lines[:5]

['149474\t139836\t2023-05-24 07:47:53\t9745590 9748574 9748432 9748080 9750687 9750802 9750829 9750793 9750726 9748041 4800276 9757533 9760252 9760528 9760272 9760067 9765336 9765185 9765156\t9778623-0 9778682-0 9778669-0 9778657-1 9778736-0 9778728-0\n',
 '150528\t143471\t2023-05-24 07:33:25\t9737881 9738659 9738569 9738490 9738528 9737521 9736689 9738760 9738684 9733713 9736693 9738569 9737393 9738746 9738303 9738902 9739035 9738968 9739088 9739065 9738980 9739042 9735495 9737243 9736662 9739411 9738974 9739499 9739443 9739634 9739072 9739640 9736767 9739864 9739471 9739293 9735311 9739864 9739883 9739553 9740003 9739999 9739967 9735579 9735579 9735579 9740427 9740223 9740013 9740432 9740400 9739938 9740843 9740710 9740662 9740570 9740962 9740957 9741428 9741449 9741124 9741419 9741108 9729924 9737083 9741508 9741476 9741360 9741311 9741415 9741523 9738585 9739272 9741788 9741772 9742146 9742014 9741896 9738844 9740324 9740711 9667501 9742801 9742770 9742735 9735166 9742327 9740576 9

In [16]:
y = 0
n = 0

for l in tqdm(lines, desc=f"Processing behaviors news to News Graph"):
    # print("lines", line)
    line = l.strip().split('\t')

    # check duplicate user
    used_id = line[1]
    if used_id in user_set:
        continue
    else:
        user_set.add(used_id)

    
    # record cnt & read path
    history = line[3].split()
    if len(history) > 1:
        # print("history1", history)
        for news_id in history:
            if int(news_id) in art_dict:
                y += 1
                # print(f'yes: {news_id}')
            else:
                n += 1
                # print(f'no: {news_id}')
        # long_edge = [art_dict[news_id] for news_id in history]
        # print(long_edge)
        # edge_list.append(long_edge)

    # print("history2", history)

y, n

Processing behaviors news to News Graph: 100%|██████████| 232887/232887 [00:01<00:00, 135442.77it/s]


(2426247, 0)

In [1]:

# Testing
import os.path
from pathlib import Path

import hydra
import wandb
import numpy as np
import torch
from omegaconf import DictConfig, OmegaConf
from torch.cuda import amp
import torch.multiprocessing as mp
import torch.distributed as dist
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR

from dataload.data_load import load_data
from dataload.data_preprocess import prepare_preprocessed_data
from utils.metrics import *
from utils.common import *

### custom your wandb setting here ###
# os.environ["WANDB_API_KEY"] = ""
# os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_API_KEY"] = "4fbf74f08e1faaadbc3bcd8d184d16f04338332b"

def train(model, optimizer, scaler, scheduler, dataloader, local_rank, cfg, early_stopping):
    model.train()
    torch.set_grad_enabled(True)

    sum_loss = torch.zeros(1).to(local_rank)
    sum_auc = torch.zeros(1).to(local_rank)
    
    train_dataset_len = len(dataloader)
    print('train : len(dataloader)', len(dataloader))
    for cnt, batch in enumerate(dataloader, start=1):
        print(f"Batch {cnt} received from dataloader")
        if cnt == 1:
            break
    print("Dataloader check completed.")

    res = val(model, local_rank, cfg)
    model.train()

    if local_rank == 0:
        save_model(cfg, model, optimizer, f"{cfg.ml_label}_auc_best")
        wandb.run.summary.update({"best_auc": res["auc"],"best_mrr":res['mrr'], 
                            "best_ndcg5":res['ndcg5'], "best_ndcg10":res['ndcg10']})

        pretty_print(res)
        wandb.log(res)


    # for cnt, (subgraph, mapping_idx, candidate_news, candidate_entity, entity_mask, labels) \
    #     in enumerate(tqdm(dataloader, total=int(train_dataset_len), desc=f"[{local_rank}] Training"), start=1):
    #     # print(f"Batch {cnt} - Data received from dataloader")

    #     subgraph = subgraph.to(local_rank, non_blocking=True)
    #     mapping_idx = mapping_idx.to(local_rank, non_blocking=True)
    #     candidate_news = candidate_news.to(local_rank, non_blocking=True)
    #     labels = labels.to(local_rank, non_blocking=True)
    #     candidate_entity = candidate_entity.to(local_rank, non_blocking=True)
    #     entity_mask = entity_mask.to(local_rank, non_blocking=True)

    #     with amp.autocast():
    #         bz_loss, y_hat = model(subgraph, mapping_idx, candidate_news, candidate_entity, entity_mask, labels)

            
    #     # Accumulate the gradients
    #     scaler.scale(bz_loss).backward()
    #     if cnt % cfg.accumulation_steps == 0 or cnt == int(train_dataset_len / cfg.batch_size):
    #         # Update the parameters
    #         scaler.step(optimizer)
    #         old_scaler = scaler.get_scale()
    #         scaler.update()
    #         new_scaler = scaler.get_scale()
    #         if new_scaler >= old_scaler:
    #             scheduler.step()
    #             ## https://discuss.pytorch.org/t/userwarning-detected-call-of-lr-scheduler-step-before-optimizer-step/164814
    #         optimizer.zero_grad(set_to_none=True)

    #     sum_loss += bz_loss.data.float()
    #     sum_auc += area_under_curve(labels, y_hat)
    #     # ---------------------------------------- Training Log
    #     if cnt % cfg.log_steps == 0:
    #         if local_rank == 0:
    #             wandb.log({"train_loss": sum_loss.item() / cfg.log_steps, "train_auc": sum_auc.item() / cfg.log_steps})
    #         print('[{}] Ed: {}, average_loss: {:.5f}, average_acc: {:.5f}'.format(
    #             local_rank, cnt * cfg.batch_size, sum_loss.item() / cfg.log_steps, sum_auc.item() / cfg.log_steps))
    #         sum_loss.zero_()
    #         sum_auc.zero_()
        
    #     if cnt % cfg.val_steps == 0:
    #         res = val(model, local_rank, cfg)
    #         model.train()

    #         if local_rank == 0:
    #             save_model(cfg, model, optimizer, f"{cfg.ml_label}_auc_best")
    #             wandb.run.summary.update({"best_auc": res["auc"],"best_mrr":res['mrr'], 
    #                                 "best_ndcg5":res['ndcg5'], "best_ndcg10":res['ndcg10']})

    #             pretty_print(res)
    #             wandb.log(res)

    #         early_stop, get_better = early_stopping(res['auc'])
    #         if early_stop:
    #             print("Early Stop.")
    #             break
    #         elif get_better:
    #             print(f"Better Result!")
    #             if local_rank == 0:
    #                 save_model(cfg, model, optimizer, f"{cfg.ml_label}_auc{res['auc']}")
    #                 wandb.run.summary.update({"best_auc": res["auc"],"best_mrr":res['mrr'], 
    #                                      "best_ndcg5":res['ndcg5'], "best_ndcg10":res['ndcg10']})



def val(model, local_rank, cfg):
    model.eval()
    dataloader = load_data(cfg, mode='val', model=model, local_rank=local_rank)
    val_dataset_len = len(dataloader)
    print('val : len(dataloader)', len(dataloader))
    tasks = []
    print('gpus', cfg.gpu_num)
    with torch.no_grad():
        for cnt, (subgraph, mappings, clicked_entity, candidate_input, candidate_entity, entity_mask, labels) \
                in enumerate(tqdm(dataloader,
                                  total=int(val_dataset_len),
                                  desc=f"[{local_rank}] Validating")):
            candidate_emb = torch.FloatTensor(np.array(candidate_input)).to(local_rank, non_blocking=True)
            candidate_entity = candidate_entity.to(local_rank, non_blocking=True)
            entity_mask = entity_mask.to(local_rank, non_blocking=True)
            clicked_entity = clicked_entity.to(local_rank, non_blocking=True)

            scores = model.module.validation_process(subgraph, mappings, clicked_entity, candidate_emb, candidate_entity, entity_mask)
            
            tasks.append((labels.tolist(), scores))

    with mp.Pool(processes=cfg.num_workers) as pool:
        results = pool.map(cal_metric, tasks)
    val_auc, val_mrr, val_ndcg5, val_ndcg10 = np.array(results).T

    # barrier
    torch.distributed.barrier()

    reduced_auc = reduce_mean(torch.tensor(np.nanmean(val_auc)).float().to(local_rank), cfg.gpu_num)
    reduced_mrr = reduce_mean(torch.tensor(np.nanmean(val_mrr)).float().to(local_rank), cfg.gpu_num)
    reduced_ndcg5 = reduce_mean(torch.tensor(np.nanmean(val_ndcg5)).float().to(local_rank), cfg.gpu_num)
    reduced_ndcg10 = reduce_mean(torch.tensor(np.nanmean(val_ndcg10)).float().to(local_rank), cfg.gpu_num)

    res = {
        "auc": reduced_auc.item(),
        "mrr": reduced_mrr.item(),
        "ndcg5": reduced_ndcg5.item(),
        "ndcg10": reduced_ndcg10.item(),
    }
    
    return res

def test(model, local_rank, cfg):
    model.eval()
    dataloader = load_data(cfg, mode='test', model=model, local_rank=local_rank)
    val_dataset_len = len(dataloader)
    print('test : len(dataloader)', len(dataloader))
    tasks = []
    print('gpus', cfg.gpu_num)
    with torch.no_grad():
        for cnt, (subgraph, mappings, clicked_entity, candidate_input, candidate_entity, entity_mask, labels) \
                in enumerate(tqdm(dataloader,
                                  total=int(val_dataset_len),
                                  desc=f"[{local_rank}] Testing")):
            candidate_emb = torch.FloatTensor(np.array(candidate_input)).to(local_rank, non_blocking=True)
            candidate_entity = candidate_entity.to(local_rank, non_blocking=True)
            entity_mask = entity_mask.to(local_rank, non_blocking=True)
            clicked_entity = clicked_entity.to(local_rank, non_blocking=True)

            scores = model.module.validation_process(subgraph, mappings, clicked_entity, candidate_emb, candidate_entity, entity_mask)
            
            tasks.append((labels.tolist(), scores))

    with mp.Pool(processes=cfg.num_workers) as pool:
        results = pool.map(cal_metric, tasks)
    val_auc, val_mrr, val_ndcg5, val_ndcg10 = np.array(results).T

    # barrier
    torch.distributed.barrier()

    reduced_auc = reduce_mean(torch.tensor(np.nanmean(val_auc)).float().to(local_rank), cfg.gpu_num)
    reduced_mrr = reduce_mean(torch.tensor(np.nanmean(val_mrr)).float().to(local_rank), cfg.gpu_num)
    reduced_ndcg5 = reduce_mean(torch.tensor(np.nanmean(val_ndcg5)).float().to(local_rank), cfg.gpu_num)
    reduced_ndcg10 = reduce_mean(torch.tensor(np.nanmean(val_ndcg10)).float().to(local_rank), cfg.gpu_num)

    res = {
        "auc": reduced_auc.item(),
        "mrr": reduced_mrr.item(),
        "ndcg5": reduced_ndcg5.item(),
        "ndcg10": reduced_ndcg10.item(),
    }
    
    return res


def main_worker(local_rank, cfg):
    # -----------------------------------------Environment Initial
    seed_everything(cfg.seed)
    dist.init_process_group(backend='nccl',
                            init_method='tcp://127.0.0.1:23456',
                            world_size=cfg.gpu_num,
                            rank=local_rank)

    # -----------------------------------------Dataset & Model Load
    num_training_steps = int(cfg.num_epochs * cfg.dataset.pos_count / (cfg.batch_size * cfg.accumulation_steps))
    num_warmup_steps = int(num_training_steps * cfg.warmup_ratio + 1)
    train_dataloader = load_data(cfg, mode='train', local_rank=local_rank)
    model = load_model(cfg).to(local_rank)
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.optimizer.lr)

    lr_lambda = lambda step: 1.0 if step > num_warmup_steps else step / num_warmup_steps
    scheduler = LambdaLR(optimizer, lr_lambda)
    
    # ------------------------------------------Load Checkpoint & optimizer
    if cfg.load_checkpoint:
        file_path = Path(f"{cfg.path.ckp_dir}/{cfg.model.model_name}_{cfg.dataset.dataset_name}_{cfg.load_mark}.pth")
        checkpoint = torch.load(file_path, map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'])  # After Distributed
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    print("dist!!!")
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank])
    optimizer.zero_grad(set_to_none=True)
    scaler = amp.GradScaler()

    # ------------------------------------------Main Start
    early_stopping = EarlyStopping(cfg.early_stop_patience)

    if local_rank == 0:
        wandb.init(config=OmegaConf.to_container(cfg, resolve=True),
                   project=cfg.logger.exp_name, name=cfg.logger.run_name)
        print(model)

    print("train")
    # for _ in tqdm(range(1, cfg.num_epochs + 1), desc="Epoch"):
    train(model, optimizer, scaler, scheduler, train_dataloader, local_rank, cfg, early_stopping)


    if local_rank == 0:
        wandb.finish()


@hydra.main(version_base="1.2", config_path=os.path.join(get_root(), "configs"), config_name="EB")
def main(cfg: DictConfig):
    seed_everything(cfg.seed)
    cfg.gpu_num = torch.cuda.device_count()
    # prepare_preprocessed_data(cfg)
    mp.spawn(main_worker, nprocs=cfg.gpu_num, args=(cfg,))


if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named 'dataload'